# Calling JS mermaid from python, in a notebook

## Demo

In [1]:
import uuid

_mermaid_display_template = """
<script type="module">
    async function initMermaid_{unique_id}() {{
        const module = await import('https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs');
        const mermaid = module.default;
        mermaid.initialize({{ startOnLoad: true }});
        mermaid.init(undefined, document.getElementById('mermaid_{unique_id}'));
    }}
    
    if (document.readyState === 'loading') {{
        document.addEventListener('DOMContentLoaded', initMermaid_{unique_id});
    }} else {{
        initMermaid_{unique_id}();
    }}
</script>

<pre class="mermaid" id="mermaid_{unique_id}">
    {code}
</pre>
"""


def mermaid(code: str, *, prefix='graph', suffix='', egress=None):
    if egress is None:
        egress = lambda x: x
    code_ = f"{prefix}\n{code}\n{suffix}"
    unique_id = str(uuid.uuid4()).replace("-", "")
    html = _mermaid_display_template.format(code=code_, unique_id=unique_id)
    return egress(html)


In [2]:
code = "A --> B --> C"

In [5]:
html = mermaid(code)
assert isinstance(html, str)
# print(html)

You can define an `egress` to tell the function to return the result of `egress(html)` instead. This is useful, for example, to save the html, or wrap it in an object that can render it. For example, `IPython.display.HTML` will wrap it in a displayable `HTML` instance.

In [6]:
from IPython.display import HTML

mermaid(code, egress=HTML)

`mermaid` is setup to be a framework through `functools.partial`. 
For example, to get a function that does left-to-right graphs (the default is top-to-bottom),
and returning an `HTML` instance systematically, you can do the following:

In [8]:
from functools import partial
from IPython.display import display, HTML

lr_graph = partial(mermaid, prefix='graph LR', egress=HTML)

In [9]:
lr_graph(code)